In [2]:
import re
import csv
import pandas as pd
import numpy as np
import configparser

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, GlobalMaxPool1D, Bidirectional, Embedding, Dropout
from keras.models import Model

# gensim
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

# sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 

#nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

## Chargement des données

In [3]:
config = configparser.ConfigParser()
config.read('../config.cfg')
EMBEDDING_FILE=config['FILES']['WORD']
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [4]:
# Chargement des données sous forme de dataframes pandas
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

# Définition des différents labels disponibles sous forme de liste
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

## Tokenisation des commentaires pour le modèle Word2vec

In [20]:
# pour créer un modèle Word2vec, il faut transformer le texte en liste de mots (tokens). 
# On retire les stopWords et la ponctuaction pour essayer de coller au modèle Google
sentences = []
stopWords = set(stopwords.words('english'))
all_comment = list_sentences_train.tolist() + list_sentences_test.tolist()
for comment in all_comment:
    comment = ' '.join(re.split('\n',comment.lower()))
    comment = re.sub(r'[^a-zA-Z0-9\s:]', '', comment)
    words = word_tokenize(comment)
    wordsFiltered = []
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    sentences.append(wordsFiltered)

In [11]:
# Pour maximiser le vocabulaire du modèle, on utilise le jeu d'entrainement et test
len(sentences)

312735

## Création du modèle Word2vec

In [26]:
# On choisit une taille de vecteur 300 comme le modèle Google
model = Word2Vec(sentences, size=300)

In [27]:
word_vectors = model.wv

### Sauvegarde et chargement du modèle Word2vec

In [28]:
fname = get_tmpfile("vectors.kv")
word_vectors.save(fname)
word_vectors = KeyedVectors.load(fname, mmap='r')

In [29]:
# get the most common words
print(word_vectors.index2word[0:20])

[':', 'article', 'page', 'wikipedia', 'would', 'like', 'one', 'talk', 'please', 'dont', 'see', 'think', 'im', 'also', 'know', 'fuck', 'people', 'articles', 'edit', 'use']


In [30]:
embed_size = 300 # taille du vecteur
max_features = 20000 # nombre de mots uniques utilisés (i.e nombre de lignes dans le vecteur d'embedding)
maxlen = 100 # nombre maximum de mots à considérer dans un commentaire

## Tokenisation pour la classification

In [31]:
# A des fins de comparaison, on utilise le même tokenizer que pour le notebook Google

# On garde les mots les plus fréquents dans le jeu d'entrainement pour établir notre liste de tokens.
tokenizer = Tokenizer(num_words=max_features)
# Calcul des mots les plus fréquents
tokenizer.fit_on_texts(list(list_sentences_train))
# Indexation des jeux d'entrainement et test, conversion du texte en séquence d'indexes
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [32]:
# Les commentaires ont une longueur variable qui peut être inférieure à maxlen
# On complète donc la séquence avec des 0, par défaut au début de la séquence
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [33]:
word_index = tokenizer.word_index
# le modèle pré-entrainé contient 3 millions de mots contre 20000 pour notre jeu de données.
vocabulary_size=min(len(word_index)+1,max_features)

# la matrice d'embedding représente le vocabulaire final à utiliser. 
# On ne gardera que 20000 vecteurs du modèle pré-entrainé. Leur longueur est de 300.
embedding_matrix = np.zeros((vocabulary_size, embed_size))


for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector    
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

### Modèle

In [37]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrainement

In [38]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 739s 5ms/step - loss: 0.0531 - acc: 0.9809 - val_loss: 0.0454 - val_acc: 0.9831
Epoch 2/2
143613/143613 [==============================] - 735s 5ms/step - loss: 0.0417 - acc: 0.9840 - val_loss: 0.0455 - val_acc: 0.9831


In [39]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [40]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 53s 832us/step


[0.07073896178930458, 0.9705133489816529]

In [41]:
y_pred_t = model.predict(X_te, batch_size=1024)

In [42]:
print(accuracy_score(yt, y_pred_t[test_label_strip.index].round()))

0.873612804401513


In [43]:
# Classification pour chaque label de façon indépendante
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt[:,label], y_pred_t[test_label_strip.index, label].round())
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt[:,label], y_pred_t[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt[:,label], y_pred_t[test_label_strip.index, label].round()) )

... Traitement du label toxic
L'accuracy du jeu test est 0.9177686079589858
Matrice de confusion sur le jeu test:
[[53443  4445]
 [  816  5274]]
Rapport : 
              precision    recall  f1-score   support

           0       0.98      0.92      0.95     57888
           1       0.54      0.87      0.67      6090

   micro avg       0.92      0.92      0.92     63978
   macro avg       0.76      0.89      0.81     63978
weighted avg       0.94      0.92      0.93     63978

... Traitement du label severe_toxic
L'accuracy du jeu test est 0.9934352433649067
Matrice de confusion sur le jeu test:
[[63464   147]
 [  273    94]]
Rapport : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.39      0.26      0.31       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.69      0.63      0.65     63978
weighted avg       0.99      0.99      0.99     63978

... Traitement du label obs